In [2]:
# Library Management System.

from datetime import datetime, timedelta
    
class Book:
    '''This is to keep records of books in the library
    It has total of 2 module: "BORROW BOOKS", "RETURN BOOKS".'''
    
    def __init__(self, title, author, isbn):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.available = True
        self.borrowed_by = None         # Tracking which user borrowed the book.
        self.due_date = None            # Settting due date


    def borrow(self, user, loan_period = 14):
        if self.available:
            self.available = False
            self.borrowed_by = user.user_id           # Store user ID.
            self.due_date = datetime.now() + timedelta(days = loan_period)
            
            time_stamp = datetime.now().strftime('%d/%m/%y %H:%M:%S')         # Track date and time the book was borrowed.
            due_date_str = self.due_date.strftime('%d/%m/%y %H:%M:%S')
            
            return f'\n{time_stamp} {user.name} borrowed {self.title} by {self.author} (ISBN: {self.isbn}).'\
                   f'\nDue Date: {due_date_str}' 
            
        else:
            return f'\nSorry {self.title} by {self.author} (ISBN: {self.isbn}) is not borrowed by user {self.borrowed_by}.'

    
    def return_book(self, user):
        if not self.available and self.borrowed_by == user.user_id:
            self.available = True
            self.borrowed_by = None     # Reset borrower info.
            
            time_stamp = datetime.now().strftime('%d/%m/%y %H:%M:%S')     # Track date and time a borrowed book was returned
            
            if datetime.now() > self.due_date:
                overdue_days = (datetime.now() - self.due_date).days
                fine = overdue_days * 200        # Multiply days by fine amount
                
                return f'\n{user.name} you are {overdue_days} passed due date.'\
                       f'\nYou will be fined kshs 200 per day.'
            
            print(f'Fine: kshs {fine}')

            return f'\n{time_stamp} {user.name} ID: {user.user_id} returned {self.title} by {self.author} (ISBN: {self.isbn}).'
        
        elif self.available:
            return f'\n{self.title} is still available in the library.'
        
        else:
            return f'\nError: {self.title} was not borrowed by  {user.name}'



class User:
    '''This is to keep the details of library user.
    There ID, name and phone number. To help keep detailed library operations.
    '''
    def __init__(self, user_id, name, phone_no):
        self.user_id = user_id
        self.name = name
        self.phone_no = phone_no
    
    def __str__(self):
        return f'ID: {self.user_id}, Name: {self.name}, Phone No: {self.phone_no}'

class Library:
    '''This is to keep library records on how books are handled.
    It has a total of 5 modules: "ADD BOOKS", "DISPLAY BOOKS", "LEND BOOK", "RETURN BOOK", "SEARCH BOOK".
    We also have library users: "ADD USERS", "LIST USERS", "UPDATE USER", "DELETE USER".'''

    def __init__(self):
        self.books = []
        self.users = {}
    
    def add_books(self, book):
        self.books.append(book)

    def display_book(self):
        available_books = [f'\n{book.title} by {book.author} (ISBN: {book.isbn})' for book in self.books if book.available]

        if available_books:
            return 'Available books in the library: \n' + '\n' .join(available_books)

        else:
            return 'No books available in the library.'

    def lend_book(self, title, isbn, user_id):
        if user_id not in self.users:
            return f'User with ID {user_id} not found.'
        
        user = self.users[user_id]    # Get the user instance.
        
        for book in self.books:
            if book.title == title and book.isbn == isbn and book.available:
                return book.borrow(user, loan_period = 14)    # Pass the user instance
            
        return f'Sorry, {title} ; {isbn} is not available in the library.'
    
    def return_book(self, title, isbn, user_id):
        if user_id not in self.users:
            return f'User with ID {user_id} not found.'
        
        user = self.users[user_id]
        
        for book in self.books:
            if book.title == title and book.isbn == isbn and not book.available:
                return book.return_book(user)
            
        return f'{title} (ISBN: {isbn}) was not borrowed from the library.'

    def search_book(self, title, author, isbn):
        for book in self.books:
            if book.title == title and book.author == author and book.isbn == isbn:
                return f'\nFound: {title} by {author} (ISBN: {isbn}). Available: {"Yes" if book.available else "No"}'

        return f'\n{title} by {author} (ISBN: {isbn}) not available.'
    
    def add_user(self, user):
        '''This is definition to add a new library user.'''

        if user.user_id in self.users:
            print(f'\n(ID: {user.user_id}) already exists.')
            return
        
        self.users[user.user_id] = user
        print(f'\n(Name: {user.name}) (ID: {user.user_id}) added.')
        
    def list_users(self):
        if not self.users:
            print('\nNo users registered.')
            return
        
        for user_id, user in self.users.items():
            print(user)

    def update_user(self, user_id, name = '', phone_no = ''):
        if user_id not in self.users:
            print('User not found.')
        
        if name:
            self.users[user_id].name = name
        if phone_no:
            self.users[user_id].phone_no = phone_no
        print(f'\nUser {user_id} updated.')
    
    def delete_user(self, user_id):
        if user_id in self.users:
            del self.users[user_id]
            print(f'\nID: {user_id} deleted')
        
        else:
            print('ID not found.')



# Creating user interface.
def main():
    manager = Library()
    
    while True:
        print('\nLibrary Management System.')
        print('\n1. Books')
        print('2. Users')
        print('3. Exit')
        
        choice = input('\nEnter your choice: ')
        
        if choice == '1':
            print('\n(a) Add book')
            print('(b) Display book')
            print('(c) Lend book')
            print('(d) Return book')
            print('(e) Search book')
            print('(f) Go back')

            choice = input('\nEnter your choice: ')
            
            if choice == 'a':
                title = input('\nEnter book title: ')
                author = input('Enter book author: ')
                isbn = input('Enter book ISBN number: ')
                manager.add_books(Book(title, author, isbn))

            elif choice == 'b':
                print(manager.display_book())
            
            elif choice == 'c':
                title = input('\nEnter book title: ')
                isbn = input('\nEnter ISBN: ')
                user_id = input('\nEnter user ID: ')    # Ask for borrowers user ID
                print(manager.lend_book(title, isbn, user_id))

            elif choice == 'd':
                title = input('\nEnter book title: ')
                isbn = input('\nEnter ISBN: ')
                user_id = input('\nEnter user ID: ')   # Ask for borrower's user ID
                print(manager.return_book(title, isbn, user_id))
            
            elif choice == 'e':
                title = input('\nEnter book title: ')
                author = input('\nEnter book author: ')
                isbn = input('\nISBN: ')
                print(manager.search_book(title, author, isbn))
            
            elif choice == 'f':
                continue

            else:
                print('\nInvalid choice!')

        elif choice == '2':
            print('\n(a) Add user')
            print('(b) List user')
            print('(c) Update user')
            print('(d) Delete user')
            print('(e) Go back')

            choice = input('\nEnter choice in 2: ')

            if choice == 'a':
                user_id = input('\nEnter user ID: ')
                name = input('Enter name: ')
                phone_no = input('Enter phone number: ')
                manager.add_user(User(user_id, name, phone_no))
            
            elif choice == 'b':
                manager.list_users()
            
            elif choice == 'c':
                user_id = input('\nEnter user ID: ')
                name = input('\nEnter name(Optional): ')
                phone_no  = input('\nEnter Phone Number(Optional): ')
                manager.update_user(user_id, name, phone_no)
            
            elif choice == 'd':
                user_id = input('\nEnter user ID: ')
                manager.delete_user(user_id)
            
            elif choice == 'e':
                continue
            
            else:
                print('\nInvalid choice!')
        
        elif choice == '3':
            print('\nExiting Library Management System. Goodbye!')
            break

        else:
            print('\nInvalid choice!')
            

                
if __name__ == "__main__":
    main()


Library Management System.

1. Books
2. Users
3. Exit

Enter your choice: 3

Exiting Library Management System. Goodbye!
